In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\Sarah Jr\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
#Set URL for NASA Mars News
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [21]:
#Create a BeautifulSoup object
html = browser.html
soup = bs(html, 'html.parser')

#Extract titles
titles = soup.find_all(name='div', class_='content_title')
#Create list comprehension 
title_list = [t.text for t in titles]
#Set index for article title and store into variable
news_title = title_list[1]

#Extract paragraphs
paragraphs = soup.find_all(name='div', class_='article_teaser_body')
#Create list comprehension 
paragraph_list= [p.text for p in paragraphs]
#Set index for article teaser paragraph and store into variable
news_p = paragraph_list[0]

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

In [ ]:
browser.quit()